# Generalized eigenvalue problem

Weak form of small-displacement free oscillations has following form

\begin{equation}
\int_\Omega\boldsymbol{\sigma}:\delta\boldsymbol{\varepsilon}\ \mathrm{d}\Omega + 
\int_\Omega\rho\ddot{\mathbf{u}}\cdot\delta\mathbf{u}\,\mathrm{d}\Omega
 = 0, \forall \delta\mathbf{u}.
\end{equation}

If we are looking for normal modes, we assume that all points of continuum oscillates with the same frequency. Therefore we can assume that displacement field is real part of following phasor

\begin{equation}
\mathbf{u}(\mathbf{x})=\mathbf{u}_A(\mathbf{x})\mathrm{e}^{\mathrm{i}\omega t}
\end{equation}

and consequently

\begin{equation}
\ddot{\mathbf{u}}(\mathbf{x})=-\omega^2\mathbf{u}_A(\mathbf{x})\mathrm{e}^{\mathrm{i}\omega t}.
\end{equation}

Using these definitions, our problem becomes

\begin{equation}
\int_\Omega\boldsymbol{\sigma}:\delta\boldsymbol{\varepsilon}\ \mathrm{d}\Omega  - 
\omega^2\int_\Omega\rho\mathbf{u}\cdot\delta\mathbf{u}\,\mathrm{d}\Omega
 = 0, \forall \delta\mathbf{u}.
\end{equation}

This is generalized eigenvalue problem, which can not be solved by default FEniCS solver. For this problems FEniCS provides class *SLEPcEigenSolver* and also FEniCS tools can be used for assembling governing matrices. In matrices notation, eigenvalue problem becomes 

\begin{equation}
\mathbf{K}\mathbf{u} - \omega^2\mathbf{M}\mathbf{u} = \mathbf{0}
\end{equation}

or 

\begin{equation}
\mathbf{M}^{-1}\mathbf{K}\mathbf{u} - \omega^2\mathbf{u} = \mathbf{0}.
\end{equation}

### Implementation

Suppose plate modelled in 3D as box with no Dirichlet boundary condition. First half of code is the same as implementation of elasticity.

>JS: Pridat boundary condition

In [2]:
import fenics as fe
import matplotlib.pyplot as plt


# --------------------
# Functions and classes
# --------------------
def bottom(x, on_boundary):
    return (on_boundary and fe.near(x[1], 0.0))


# Strain function
def epsilon(u):
    return 0.5*(fe.nabla_grad(u) + fe.nabla_grad(u).T)


# Stress function
def sigma(u):
    return lmbda*fe.div(u)*fe.Identity(3) + 2*mu*epsilon(u)


# --------------------
# Parameters
# --------------------
# Young modulus, poisson number and density
E, nu = 70.0E9, 0.23
rho = 2500.0

# Lame's constants
mu = E/2./(1+nu)
lmbda = E*nu/(1+nu)/(1-2*nu)

l_x, l_y, l_z = 1.0, 1.0, 0.01  # Domain dimensions
n_x, n_y, n_z = 20, 20, 2  # Number of elements

# --------------------
# Geometry
# --------------------
mesh = fe.BoxMesh(fe.Point(0.0, 0.0, 0.0), fe.Point(l_x, l_y, l_z), n_x, n_y, n_z)

# --------------------
# Boundary conditions
# --------------------
bc = []

# --------------------
# Function spaces
# --------------------
V = fe.VectorFunctionSpace(mesh, "CG", 2)
u_tr = fe.TrialFunction(V)
u_test = fe.TestFunction(V)

First of all, we must define corresponding weak forms of integral using trial and test function. 

In [3]:
a_form = fe.inner(sigma(u_tr), epsilon(u_test))*fe.dx
m_form = rho*fe.inner(u_tr, u_test)*fe.dx

Now we can create two *PETScMatrix()* objects. PETSc provides creating and manipulating with sparse matrices. Last two lines assemble stiffness and mass matrix and stores them to objects *A* and *M*. There is power of FEniCS, because by calling *fe.assemble()*, FEniCS creates matrix, performs integration and localization with no effort.

In [4]:
a_form = fe.inner(sigma(u_tr), epsilon(u_test))*fe.dx
m_form = rho*fe.inner(u_tr, u_test)*fe.dx

A = fe.PETScMatrix()
M = fe.PETScMatrix()
A = fe.assemble(a_form, tensor=A)
M = fe.assemble(m_form, tensor=M)

Solver is created by calling *SLEPcEigenSolver* constructor with two arguments: stiffness matrix *A* and mass matrix *M*. Following lines provides basic settings of solver. These tables briefly introduce these parameters:

| "problem_type" | Description |
| -- | -- |
| “hermitian” | Hermitian |
| “non_hermitian” | Non-Hermitian |
| “gen_hermitian” | Generalized Hermitian |
| “gen_non_hermitian” | Generalized Non-Hermitian |
| “pos_gen_non_hermitian” | Generalized Non-Hermitian with positive semidefinite M |

| "spectrum" | Description |
| -- | -- |
| “largest magnitude” | eigenvalues with largest magnitude |
| “smallest magnitude” | eigenvalues with smallest magnitude |
| “largest real” | eigenvalues with largest double part |
| “smallest real” | eigenvalues with smallest double part |
| “largest imaginary” | eigenvalues with largest imaginary part |
| “smallest imaginary” | eigenvalues with smallest imaginary part |
| “target magnitude” | eigenvalues closest to target in magnitude (versions >= 3.1) |
| “target real” | eigenvalues closest to target in real part (versions >= 3.1) |
| “target imaginary” | eigenvalues closest to target in imaginary part (versions >= 3.1) |

| "spectral_transform" | Description |
| -- | -- |
| “shift-and-invert” | A shift-and-invert transform |
| " " | No spectral transform |

More parameters are presented in https://fenicsproject.org/docs/dolfin/1.4.0/python/programmers-reference/cpp/la/SLEPcEigenSolver.html.

In [6]:
# --------------------
# Eigen-solver
# --------------------
eigensolver = fe.SLEPcEigenSolver(A, M)
eigensolver.parameters["problem_type"] = "gen_hermitian"
eigensolver.parameters["spectrum"] = "smallest real"
eigensolver.parameters["spectral_transform"] = "shift-and-invert"
eigensolver.parameters["spectral_shift"] = 100.0

Now we can solve the eigenvalue problem. Solver takes one parameter: number of eigenvalues to be calculated.

In [8]:
N_eig = 12   # number of eigenvalues
eigensolver.solve(N_eig)

The solution is prepared in object *eigensolver* and we want extract it and save to *.xdmf* file. Therefore we create *XDMFFile()* with some additional parameters. `flush_output` with value true can allow inspection of results while process is still running. Parameter `functions_share_mesh` optimize saving process considering that all functions have the same mesh.

In [9]:
# --------------------
# Post-process
# --------------------
# Set up file for exporting results
file_results = fe.XDMFFile("MA.xdmf")
file_results.parameters["flush_output"] = True
file_results.parameters["functions_share_mesh"] = True

The last part of code calculate for each eigenvalue corresponding eigenfrequency and save eigenvector as fenics `Function`.

In [10]:
# Eigenfrequencies
for i in range(0, N_eig):
    # Get i-th eigenvalue and eigenvector
    # r - real part of eigenvalue
    # c - imaginary part of eigenvaue
    # rx - real part of eigenvector
    # cx - imaginary part of eigenvector
    r, c, rx, cx = eigensolver.get_eigenpair(i)
    
    # Calculation of eigenfrequency from real part of eigenvalue
    freq_3D = fe.sqrt(r)/2/fe.pi
    print("Eigenfrequency: {0:8.5f} [Hz]".format(freq_3D))
    
    # Initialize function and assign eigenvector
    eigenmode = fe.Function(V, name="Eigenvector " + str(i))
    eigenmode.vector()[:] = rx
    
    # Write i-th eigenfunction to xdmf file
    file_results.write(eigenmode, i)


Eigenfrequency:  0.00814 [Hz]
Eigenfrequency:  0.00835 [Hz]
Eigenfrequency:  0.00857 [Hz]
Eigenfrequency:  0.01140 [Hz]
Eigenfrequency:  0.01184 [Hz]
Eigenfrequency:  0.01208 [Hz]
Eigenfrequency: 35.16121 [Hz]
Eigenfrequency: 50.83763 [Hz]
Eigenfrequency: 59.78057 [Hz]
Eigenfrequency: 89.80385 [Hz]
Eigenfrequency: 90.36883 [Hz]
Eigenfrequency: 153.76596 [Hz]


### Code

In [2]:
import fenics as fe
import matplotlib.pyplot as plt


# --------------------
# Functions and classes
# --------------------
def bottom(x, on_boundary):
    return (on_boundary and fe.near(x[1], 0.0))


# Strain function
def epsilon(u):
    return 0.5*(fe.nabla_grad(u) + fe.nabla_grad(u).T)


# Stress function
def sigma(u):
    return lmbda*fe.div(u)*fe.Identity(3) + 2*mu*epsilon(u)


# --------------------
# Parameters
# --------------------
# Young modulus, poisson number and density
E, nu = 70.0E9, 0.23
rho = 2500.0

# Lame's constants
mu = E/2./(1+nu)
lmbda = E*nu/(1+nu)/(1-2*nu)

l_x, l_y, l_z = 1.0, 1.0, 0.01  # Domain dimensions
n_x, n_y, n_z = 20, 20, 2  # Number of elements

# --------------------
# Geometry
# --------------------
mesh = fe.BoxMesh(fe.Point(0.0, 0.0, 0.0), fe.Point(l_x, l_y, l_z), n_x, n_y, n_z)

# --------------------
# Function spaces
# --------------------
V = fe.VectorFunctionSpace(mesh, "CG", 2)
u_tr = fe.TrialFunction(V)
u_test = fe.TestFunction(V)

# --------------------
# Forms & matrices
# --------------------
a_form = fe.inner(sigma(u_tr), epsilon(u_test))*fe.dx
m_form = rho*fe.inner(u_tr, u_test)*fe.dx

A = fe.PETScMatrix()
M = fe.PETScMatrix()
A = fe.assemble(a_form, tensor=A)
M = fe.assemble(m_form, tensor=M)

# --------------------
# Eigen-solver
# --------------------
eigensolver = fe.SLEPcEigenSolver(A, M)
eigensolver.parameters['problem_type'] = 'gen_hermitian'
eigensolver.parameters["spectrum"] = "smallest real"
eigensolver.parameters['spectral_transform'] = 'shift-and-invert'
eigensolver.parameters['spectral_shift'] = 100.0

N_eig = 12   # number of eigenvalues
eigensolver.solve(N_eig)

# --------------------
# Post-process
# --------------------
# Set up file for exporting results
file_results = fe.XDMFFile("MA.xdmf")
file_results.parameters["flush_output"] = True
file_results.parameters["functions_share_mesh"] = True

# Eigenfrequencies
for i in range(0, N_eig):
    r, c, rx, cx = eigensolver.get_eigenpair(i)
    freq_3D = fe.sqrt(r)/2/fe.pi
    print("Solid FE: {0:8.5f} [Hz]".format(freq_3D))
    
    # Initialize function and assign eigenvector (renormalize by stiffness matrix)
    eigenmode = fe.Function(V, name="Eigenvector "+str(i))
    eigenmode.vector()[:] = rx

    file_results.write(eigenmode, i)


Solid FE:  0.00814 [Hz]
Solid FE:  0.00835 [Hz]
Solid FE:  0.00857 [Hz]
Solid FE:  0.01140 [Hz]
Solid FE:  0.01184 [Hz]
Solid FE:  0.01208 [Hz]
Solid FE: 35.16121 [Hz]
Solid FE: 50.83763 [Hz]
Solid FE: 59.78057 [Hz]
Solid FE: 89.80385 [Hz]
Solid FE: 90.36883 [Hz]
Solid FE: 153.76596 [Hz]
